In [0]:
%run ../python/data_provider

In [0]:
import time
def test_data_provider(share, share_file, sharing_identifier, catalog):
  pass

#test_data_provider(share="amr_share", share_file = '/dbfs/FileStore/tables/amr_azure_share.share', sharing_identifier="", catalog="amrali_d2o")
test_data_provider(share="amr_share", share_file="", sharing_identifier='', catalog="amrali_d2d")

#test_data_provider(share_file = '/dbfs/FileStore/tables/open_datasets.share', catalog="amrali_d2o")